In [ ]:
# %pip install -qq ultralytics

### Imports

In [9]:
from ultralytics import YOLO
from ultralytics.data.dataset import YOLODataset
import requests
import zipfile
import os, glob, random, shutil
from IPython.display import Image

### Prepare dataset

In [10]:
# Define dataset URL and local file name
url = "https://www.kaggle.com/api/v1/datasets/download/valentynsichkar/traffic-signs-dataset-in-yolo-format"
zip_path = "traffic-signs-dataset.zip"
extract_folder = "traffic-signs-dataset-original"

# Download the dataset
response = requests.get(url, stream=True)
total_size = int(response.headers.get('content-length', 0))
downloaded_size = 0

if not os.path.exists(extract_folder):
    with open(zip_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024*1024):
            if chunk:
                file.write(chunk)
                downloaded_size += len(chunk)
                percentage = (downloaded_size / total_size) * 100
                print(f"Download progress: {percentage:.2f}%")

    # Extract the dataset
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_folder)

    # Remove the zip file
    os.remove(zip_path)

    print("Download and extraction completed.")

    with open(os.path.join(extract_folder, ".gitignore"), "w") as file:
        file.write("*")

else:
    print("Dataset already downloaded.")


Dataset already downloaded.


In [11]:
image_files = glob.glob(f"{extract_folder}/ts/**/*.jpg", recursive=True)

print(f"Total number of images: {len(image_files)}")

Total number of images: 741


In [ ]:
dataset_folder = "traffic-signs-dataset"
test_size = 0.2

train_folder = os.path.join(dataset_folder, "train")
test_folder = os.path.join(dataset_folder, "test")

test = random.sample(image_files, int(len(image_files) * test_size))
train = list(set(image_files) - set(test))

os.makedirs(os.path.join(train_folder, "images"), exist_ok=True)
os.makedirs(os.path.join(train_folder, "labels"), exist_ok=True)

for file in train:
    label_file = file.replace(".jpg", ".txt")
    shutil.copy(file, os.path.join(train_folder, 'images', file.split("/")[-1]))
    shutil.copy(label_file, os.path.join(train_folder, 'labels', label_file.split("/")[-1]))


os.makedirs(os.path.join(test_folder, "images"), exist_ok=True)
os.makedirs(os.path.join(test_folder, "labels"), exist_ok=True)
for file in test:
    label_file = file.replace(".jpg", ".txt")
    shutil.copy(file, os.path.join(test_folder, 'images', file.split("/")[-1]))
    shutil.copy(label_file, os.path.join(test_folder, 'labels', label_file.split("/")[-1]))


In [17]:
with open(os.path.join(dataset_folder, "dataset.yaml"), 'w') as file:
    file.write(f"train: train\n")
    file.write(f"val: test\n")
    with open(os.path.join(extract_folder, "classes.names"), 'r') as classes_file:
        classes = classes_file.readlines()
        file.write(f"nc: {len(classes)}\n")
        file.write(f"names:\n")
        for class_name in classes:
            file.write(f"  - {class_name}")
            
with open(os.path.join(dataset_folder, ".gitignore"), 'w') as file:
    file.write("*")

In [14]:
dataset_yaml_path = f"{os.path.abspath(dataset_folder)}/dataset.yaml"

### Model definition

In [15]:
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 97.6MB/s]


### Initial tests

In [16]:
model.track(
    'traffic-signs-dataset-original/traffic-sign-to-test.mp4',
    save=True, save_txt=True, save_conf=True
    )

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.3s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/56) /content/traffic-signs-dataset-original/traffic-sign-to-test.mp4: 384x640 1 stop sign, 65.8ms
video 1/1 (fram

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

### Training

In [ ]:
dataset_yaml_path = f"{os.path.abspath(dataset_folder)}/dataset.yaml"
print(dataset_yaml_path)
results = model.train(data=dataset_yaml_path, epochs=100, batch=-1, patience=10)

/content/traffic-signs-dataset/dataset.yaml
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/traffic-signs-dataset/dataset.yaml, epochs=100, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

100%|██████████| 755k/755k [00:00<00:00, 24.1MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/traffic-signs-dataset/train/labels... 593 images, 0 backgrounds, 0 corrupt: 100%|██████████| 593/593 [00:00<00:00, 1087.73it/s]

train: WARNING ⚠️ /content/traffic-signs-dataset/train/images/00340.jpg: 1 duplicate labels removed
train: New cache created: /content/traffic-signs-dataset/train/labels.cache


...

                   all        148        231      0.936      0.854      0.933      0.745
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 52, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



62 epochs completed in 0.206 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


                   all        148        231      0.898      0.843      0.932      0.751
           prohibitory         76        106      0.943       0.94      0.988      0.812
                danger         35         42          1      0.816      0.987      0.785
             mandatory         29         33      0.734      0.758      0.811      0.687
                 other         42         50      0.914       0.86      0.944      0.719
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cddc2ca8110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
shutil.copy(f'{os.path.abspath(results.save_dir)}/weights/best.pt', 'finetuned.pt')

In [19]:
finetuned = YOLO("finetuned.pt")

finetuned.track(
    'traffic-signs-dataset-original/traffic-sign-to-test.mp4',
    save=True, save_txt=True, save_conf=True
    )



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/56) /home/iohans/imd_ai/notebooks/traffic-sign-detection/traffic-signs-dataset-original/traffic-sign-to-test.mp4: 384x640 2 prohibitorys, 49.1ms
video 1/1 (frame 2/56) /home/iohans/imd_ai/notebooks/traffic-sign-detection/traffic-signs-dataset-original/traffic-sign-to-test.mp4: 384x640 2 prohibitorys, 42.8ms
video 1/1 (frame 3/56) /home/iohans/imd_ai/notebooks/traffic-sign-detection/traffic-signs-dataset-original/traffic-sign-to-test.

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'prohibitory', 1: 'danger', 2: 'mandatory', 3: 'other'}
 obb: None
 orig_img: array([[[208, 200, 195],
         [208, 200, 195],
         [208, 200, 195],
         ...,
         [146, 126, 123],
         [146, 126, 123],
         [146, 126, 123]],
 
        [[208, 200, 195],
         [208, 200, 195],
         [208, 200, 195],
         ...,
         [146, 126, 123],
         [146, 126, 123],
         [146, 126, 123]],
 
        [[208, 200, 195],
         [208, 200, 195],
         [208, 200, 195],
         ...,
         [146, 126, 123],
         [146, 126, 123],
         [146, 126, 123]],
 
        ...,
 
        [[ 38,  43,  56],
         [ 38,  43,  56],
         [ 38,  43,  56],
         ...,
         [ 24,  36,  55],
         [ 21,  33,  52],
         [ 56,  68,  87]],
 
        [[ 42,  47,  60],
         [ 42,  47,  60],
         [ 4

In [26]:
with zipfile.ZipFile("runs.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk("runs"):
        for file in files:
            zipf.write(os.path.join(root, file))